# Quantum Classical Reservoir Computing

Reservoir computing (RC) is a computational framework derived from recurrent neural networks (RNNs) that offers an efficient and flexible approach for processing temporal data. Unlike traditional RNNs, where all weights are trained, reservoir computing simplifies the learning process by keeping most of the network (i.e. the "reservoir") fixed and only training the output layer. This significantly reduces computational complexity while retaining the network's ability to model dynamic and nonlinear systems.

In RC, the reservoir itself consists of a large, sparsely connected network of nonlinear units. When time-series data is input into the system, the reservoir transforms it into a high-dimensional dynamic representation. Because of the reservoir's rich dynamics and memory, it captures temporal dependencies and underlying patterns in the data. The output layer, typically a linear readout, is then trained to map these internal states to the desired output.

RC has shown strong performance in a wide range of time-series prediction tasks, including weather forecasting, financial market modeling, speech recognition, and system control. Its ability to handle noisy and nonlinear data, combined with fast and stable training, makes it a compelling choice for real-time prediction problems and applications where interpretability and efficiency are important.

Due to training problems with quantum AI models (i.e. barren plateaus, measurement overhead), it was hypothesized that Quantum Reservoir Computing (QRC) could be beneficial. As the Hilbert space grows exponentially with the number of qubits, the dimensionality of the quantum reservoir also grows exponentially. Also, the reservoir is not trained so there is no barren plateau and the measurement overhead is substantially reduced.

## The Airbus/qBraid solution

Designing a good reservoir is difficult in general and this is especially true with quantum reservoirs. We determined that this problem stems from the difficulty in describing long- and short- term dynamics via the inherent unitary evolution of quantum computers. It is necessary to have mid-evolution measurements to have non-unitarity evolution using quantum computers but these midcircuit measurements make it difficult to cover the full spectrum (i.e. eigenvalues) of timescales. We determined that by combining multiple reservoirs with slightly different parameters, we could generate a better representation general evolutions. The ideas and the quantum circuit that we utilize is described in [arXiv:2505.22837](https://arxiv.org/abs/2505.22837) and is shown below where $f_b$ and $b$ are the only hyperparameters. $f_b$ is the feedback parameter that determines the long vs short term memory and $b$ is a property of the reservoir. $y_q =arccos (Z_q)$ where $Z_q$ is the expectation of $Z$ for qubit $q$ for the previous time-step.

In [ ]:
from qiskit.circuit.library import EfficientSU2
from qiskit import QuantumCircuit
from qiskit.circuit.parameter import Parameter

import warnings

# Ignore the DeprecationWarnings coming from qiskit EfficientSU2 that can not be avoided.
warnings.filterwarnings("ignore", category=DeprecationWarning)

def qrc_circuit(num_qubits):
    phicirc = EfficientSU2(num_qubits=num_qubits, su2_gates=["ry"], skip_final_rotation_layer=True,
                    entanglement="full", reps=1, parameter_prefix="phi")
    vcirc1 = EfficientSU2(num_qubits=num_qubits, su2_gates=["ry"], skip_final_rotation_layer=False,
                    entanglement="full", reps=1, parameter_prefix="v1")
    vcirc2 = EfficientSU2(num_qubits=num_qubits, su2_gates=["ry"], skip_final_rotation_layer=False,
                    entanglement="full", reps=1, parameter_prefix="v2")
    qc = QuantumCircuit(num_qubits, num_qubits)
    qc = qc.compose(phicirc)
    qc = qc.compose(vcirc1)
    qc = qc.compose(vcirc2)
    f_b = Parameter("f_b b x")
    zs = [Parameter(f"b y_{i}") for i in range(num_qubits)]
    vars = [f_b]+zs[:-1]+ zs*4
    qrc = qc.assign_parameters(vars)
    return qrc



num_qubits = 3
qrc_circuit(num_qubits).decompose().draw(fold=-1)

## Generate the Onion Classical Quantum Reservoir Computing (OCQRC) model

In [ ]:
%pip install -q scikit-learn
# Change to use GPU
use_gpu=False
if use_gpu:
    %pip install -q cupy-cuda12x
    import cupy as cp
    
import numpy as np
from itertools import combinations
import torch.nn as nn
from sklearn.linear_model import Ridge
from qiskit.quantum_info import SparsePauliOp, Pauli
from qiskit import transpile
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_aer.backends import AerSimulator

We now generate a torch model that uses a mixed of classical and multiple quantum reservoirs. The evolution of each individual reservoir only depends on itself but the readout layer to predict the next point depends on the output of all the coupled reservoirs.

In [ ]:
class OCQRC_Model(nn.Module):
    def __init__(self, num_qubits=0, crc_size=0, use_classical=True, use_quantum=True,
                 backends=None, b=-0.31, f_bs=[0.1], ridge_param=1.e-6, use_gpu=False):
        """The Onion Classical Quantum Reservoir Computing Model
        
        Args:
            num_qubits (int): The number of qubits for the quantum reservoir.
            crc_size(int): The size of the classical reservoir.
            use_classical(bool): Whether to include a classical reservoir
            use_quantum(bool): Whether to include the quantum reservoir(s)
            backends(List(Backend)): The number of backends determines the number of quantum reservoirs
            b(float): The b parameter for the QRC circuit
            f_bs(List[Float]): The different feedback parameters for the quantum reservoir circuit
            ridge_param(float): The regularization parameter for the ridge regression
            use_gpu(bool): If true, use qiskit_aer gpu and cupy
        """
        super().__init__()
        self.num_qubits = num_qubits
        self.crc_size = crc_size
        self.observables = self.observables_qrc_circuit()
        self.circuit = self.full_circuit()


        self.b = b
        self.f_bs = f_bs
        self.use_gpu = use_gpu
        
        self.train_results = []
        self.train_outputs = []


        self.backends = backends
        self.pms = []
        self.pmc = []
        for b in self.backends:
            self.pms += [generate_preset_pass_manager(1, backend = b)]
            self.pmc += [self.pms[-1].run(self.circuit)]
        self.last_output = [[0] for i in range(len(self.backends))]
        
        self.ridge = Ridge(ridge_param)

        # Generate initial states for CRC and QRC evolution
        self.use_classical = use_classical
        self.use_quantum = use_quantum
        if self.use_quantum:
            self.init_qrc()
        else:
            self.initial_exp_values = None
        if self.use_classical:
            self.init_crc()
        else:
            self.init_xinit = None

    def init_crc(self):
        """Initialize the Classical Reservoir"""
        # generate the ESN reservoir
        self.inSize = 1
        self.outSize = 1
        self.resSize = self.crc_size
        np.random.seed(2)
        if use_gpu:
            self.Win = (np.random.rand(self.resSize,1+self.inSize) - 0.5) * 1
            self.W = np.random.rand(self.resSize, self.resSize) - 0.5 
            
            # normalizing and setting spectral radius (correct, slow):
            rhoW = max(abs(np.linalg.eig(self.W)[0]))
            self.W *= 1.00 / rhoW
            self.init_xinit = cp.tanh(cp.ones((self.resSize, 1))/cp.sqrt(self.resSize))
            self.x = cp.array(self.init_xinit.copy())
            self.W = cp.array(self.W)
            self.Win = cp.array(self.Win)
        else:
            self.Win = (np.random.rand(self.resSize,1+self.inSize) - 0.5) * 1
            self.W = np.random.rand(self.resSize, self.resSize) - 0.5 
            
            # normalizing and setting spectral radius (correct, slow):
            rhoW = max(abs(np.linalg.eig(self.W)[0]))
            self.W *= 1.00 / rhoW
            self.init_xinit = np.tanh(np.ones((self.resSize, 1))/np.sqrt(self.resSize))
            self.x = self.init_xinit.copy()
    
    def evolve_crc(self, t0):
        """Evolve the classical reservoir given the previous time t0"""
        if self.use_classical:
            u = t0/100
            if use_gpu:
                self.x = cp.tanh( cp.dot( self.Win, cp.vstack((1,u)) ) + cp.dot( self.W, self.x ) )
                return list(np.vstack((1,u, self.x.get())).flatten())
            else:
                self.x = np.tanh( np.dot( self.Win, np.vstack((1,u)) ) + np.dot( self.W, self.x ) )
                return list(np.vstack((1,u, self.x)).flatten())
        else:
            return []

    def observables_qrc_circuit(self):
        """Generate the observables for a num_qubits quantum reservoir."""
        
        # First, all the single qubit Z_q terms
        observables = []
        for i in range(0, self.num_qubits):
            op = ["I"]*(self.num_qubits)
            op[self.num_qubits-1-i] = "Z"
            observables += [SparsePauliOp(Pauli("".join(op)))]
        # All the pairs of Z_iZ_j
        for i,j in combinations(list(range(self.num_qubits)), r=2):
                op = ["I"]*(self.num_qubits)
                op[self.num_qubits-1-i] = "Z"
                op[self.num_qubits-1-j] = "Z"
                observables += [SparsePauliOp(Pauli("".join(op)))]
        
        # Generate integer represenation of observables for quick calculations
        self._int_observables = np.zeros(len(observables), dtype=int)
        for o, obs in enumerate(observables):
            pstring = obs.paulis[0].to_instruction().params[0]
            birep = int("".join(["0" if p != "Z" else "1" for p in pstring ]), base=2)
            self._int_observables[o] = birep
        return observables
    
    def full_circuit(self):
        """Return the full qrc circuit as described in arXiv:2505.22837 """
        return qrc_circuit(self.num_qubits)

    def calc_observables(self, samples):
        """Given the samples from the quantum reservoir, calculate all observables"""
        exps = np.zeros(len(self.observables), dtype=float)
        for i, r in enumerate(samples): #.items():
            for o, obs in enumerate(self._int_observables):
                exps[o] += (-1)**bin(obs & i).count("1")*r  # /self.n_shots
        return exps

    def init_qrc(self):
        """Initial state for the quantum reservoir"""
        self.last_output = [list(np.ones(self.num_qubits)*0.5) for _ in range(len(self.backends))]
        from copy import deepcopy
        self.initial_exp_values = deepcopy(self.last_output)

    def evolve_qrc(self, t0):
        """Evolve the quantum reservoir given an input signal t0"""
        obsvs = []
        if self.use_quantum:
            for c in range(len(self.backends)):
                dt, dtu = self.b, self.f_bs[c]
                f_b = t0 * dt * dtu
                zs = self.last_output[c]
                
                params = [list(np.array(np.arccos(zs[:])) * dt) + [f_b]]
                
                rcirc = self.pmc[c].assign_parameters(params[0])
                rcirc.save_statevector()

                state = self.backends[c].run([rcirc]).result().get_statevector().data
                probs = (np.conj(state)*state).real
                obsvs_new = self.calc_observables(probs)

                self.last_output[c] = obsvs_new[:self.num_qubits]
                obsvs += list(obsvs_new)


        return list(np.array(obsvs))

    def combined(self, t0):
        """Evolve both quantum and classical and return combined data"""
        return self.evolve_qrc(t0) + self.evolve_crc(t0)
    
    def train(self, x):
        """Evolve each set of time evolutions and train on results"""
        batch_size = x.shape[0]

        for b in range(batch_size):
            exp_values = []
            output_values = []
            from copy import deepcopy
            self.last_output = deepcopy(self.initial_exp_values)
            self.x = deepcopy(self.init_xinit)

            for _ in range(2):
                _ = self.combined(x[b, 0])

            exp_values += [self.combined(x[b, 0])]

            output_values += [x[b, 1]]
            p = 1
            for value in x[b, 2:]:
                exp_values += [self.combined(output_values[-1])]
                output_values += [value]
                p+=1
            self.train_results += exp_values
            self.train_outputs += output_values
        self.fit()

    def fit(self):
        """Perform ridge regression on all the evolved training data"""
        cw = np.array(self.train_results)
        self.ridge.fit(cw, y=self.train_outputs)

    def forward(self, x, num_predict):
        """Given a set of initial evolutions, predict num_predict time steps into the future"""
        batch_size = x.shape[0]

        all_outputs = np.zeros((batch_size, num_predict))
        output_values = [0.]
        exp_values = []
        for b in range(batch_size):
            from copy import deepcopy
            self.last_output = deepcopy(self.initial_exp_values)
            self.x = deepcopy(self.init_xinit)
            
            for _ in range(2):
                _ = (self.combined(x[b, 0]))
            
            exp_values += [self.combined(x[b, 0])]

            output_values += [x[b, 1]]
            p = 1
            for value in x[b, 2:]:

                exp_values += [self.combined(output_values[-1])]
                p += 1
                output_values += [value]
        
            qrc_outputs = [self.combined(output_values[-1])]

            test_values = [self.ridge.predict(np.array(qrc_outputs[-1]).reshape(1, len(qrc_outputs[-1])))[0]]

            p += 1
            
            for _ in range(num_predict-1):

                qrc_outputs += [self.combined(test_values[-1])]
                test_values += [self.ridge.predict(np.array(qrc_outputs[-1]).reshape(1, len(qrc_outputs[-1])))[0]]
                
                p += 1
                
                
            all_outputs[b, :] = test_values
        return all_outputs

## The prediction problem

The prediction problem which we will use here is the superposition of two damped harmonic oscilators with frequencies that are irrational constants of each other (in this case $\sqrt{2}$). This means that the system is aperiodic but not chaotic. We will try and predict the evolution of a time-series given a training and test set derived from the noisy evolution of the following distribution of time-series where $N()$ is a unit normal random number.

$f(t)=30e^{-t/20}(\cos\left[\left(\frac{1}{2}+\frac{4}{10}N()\right)t\right]+\cos\left[\frac{1}{\sqrt{2}}\left(\frac{1}{2}+\frac{4}{10}N()\right)t\right]) + \frac{3}{20}N(t)$



Where quantum shines is generally in the low data regime so we are going to only use 10 training distributions to predict 20 time-steps of 4 evolutions given the first 6 points.

In [ ]:

npts = 26  # Total evolution time
npred = 20  # Number of time-steps to predict
n_train = 10  # Number of training samples
n_test = 4  # The number of test samples.

from numpy.random import Generator, PCG64
rng = Generator(PCG64(1))

# total evolution time of 26
t = np.linspace(0,26, num=npts)
oscis = 0.5 + rng.standard_normal(n_train+n_test)*0.4

train_data = np.zeros((n_train, npts))
for j in range(n_train):
    train_data[j,:] = (np.exp(-0.05*t)*(np.cos(oscis[j]*t) + np.cos(oscis[j]*t/np.sqrt(2)))*30 + rng.standard_normal(npts)*0.3)

test_data = np.zeros((n_test,npts))
for j in range(n_test):
    test_data[j,:] = np.exp(-0.05*t)*(np.cos(oscis[j+n_train]*t)+np.cos(oscis[j+n_train]*t/np.sqrt(2)))*30 + rng.standard_normal(npts)*0.3

In [ ]:
from sklearn.metrics import r2_score
# Generate simple model which just averages over the train data
simple_model = np.mean(train_data.reshape(n_train,npts)[:,npts-npred:].T, axis=1)

# Reshape test data
all_tests = test_data.reshape(n_test, npts)[:,npts-npred:]

simple_r2 = r2_score(np.array(all_tests).flatten(), np.array(list(simple_model)*n_test).flatten())
print(f"The simple model has an R^2 of {simple_r2}")

# The hyper parameters for the OCQRC model
b = -0.33
f_b = 0.11
num_qubits = 6
n_qrc_layers = 3
f_bs = [f_b, f_b*1.25, f_b*1.125]
crc_size = 850
ridge_regularization = 3.e-4

r2scores = []
all_predictions = []

if use_gpu:
    backend = AerSimulator(method='statevector', device='GPU')
else:
    backend = AerSimulator(method='statevector', device='CPU')

for use_classical, use_quantum in [(True, False), (False, True), (True, True)]:

    model = OCQRC_Model(num_qubits=num_qubits, crc_size=crc_size,
                        use_classical=use_classical, use_quantum=use_quantum,
                        backends=[backend]*n_qrc_layers,
                        b=b, f_bs=f_bs,
                        ridge_param=ridge_regularization,use_gpu=use_gpu)

    # Run through the 10 instances of training series of length 26
    model.train(train_data.reshape(n_train, npts)[:, :])

    # Run through the 2 instances of test series to predict 20 points of data given the first 6 points
    all_predictions += [model.forward(test_data.reshape(n_test, npts)[:,:-npred], npred)]
    
    r2scores += [r2_score(np.array(all_tests).flatten(), np.array(all_predictions[-1]).flatten())]
    label = "classical" if use_classical else ""
    label += "+" if use_classical and use_quantum else ""
    label += "quantum" if use_quantum else ""
    print(f'The {label} model has an R^2 of {r2scores[-1]}')

We can see the $R^2$ results of each model but let's plot the results to see how each model did for each of the four time-series!

In [ ]:
import matplotlib.pyplot as plt
# Create a 2x2 grid of subplots
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 9)) # figsize adjusts the overall figure size

for i in range(2):
    for j in range(2):
        axes[i, j].plot(all_tests.T[:,2*i+j], label="Actual")
        axes[i, j].plot(all_predictions[0].T[:,2*i+j], label='CRC')
        axes[i, j].plot(all_predictions[1].T[:,2*i+j], label='OQRC')
        axes[i, j].plot(all_predictions[2].T[:,2*i+j], label='OQCRC')
        axes[i, j].legend()

There are hyperparameters where classical beats quantum, and quantum beats classical but most of the time in this noisy low training data regime, the combined classical+quantum reservoir model performs best. 

Unlike most quantum reservoir models developed, the circuit we designed tends to improve with increasing qubit count and is not restricted to a particular set of qubits. So feel free to increase the classical reservoir size or the quantum reservoir size. At a certain point, it will definitely be advantageous to use GPUs!

## Concluding Remarks

We have provided an example based on our work that classical+quantum provides the best predictions of the time-evolution for this use case of the sum of two damped harmonic oscillators. In our paper quantum+classical also provided the best results for aerospace material degradation prediction which had the same low-quantity noisy data.

Can you find a problem where large classical/quantum reservoirs are required? Good luck!